In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
%matplotlib inline

import selenium
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import requests
import copy

In [2]:
moscow_okrug = {
    'Центральный административный округ':
 [
     'Арбат',
 'Басманный район',
 'Замоскворечье',
 'Красносельский район',
 'Мещанский район',
 'Пресненский район',
 'Таганский район',
 'Тверской район',
 'Хамовники',
 'Якиманка'],
                
 'Северный административный округ':
 [
     'Аэропорт',
 'Беговой',
 'Бескудниковский район',
 'Войковский район',
 'Восточное Дегунино',
 'Головинский район',
 'Дмитровский район',
 'Западное Дегунино',
 'Коптево',
 'Левобережный',
 'Молжаниновский район',
 'Савёловский район',
 'Сокол',
 'Тимирязевский район',
 'Ховрино',
 'Хорошёвский район'],
                
 'Северо-Восточный административный округ':
 [
     'Алексеевский район',
 'Алтуфьевский район',
 'Бабушкинский район',
 'Бибирево',
 'Бутырский район',
 'Лианозово',
 'Лосиноостровский район',
 'Марфино',
 'Марьина роща',
 'Останкинский район',
 'Отрадное',
 'Ростокино',
 'Свиблово',
 'Северный',
 'Северное Медведково',
 'Южное Медведково',
 'Ярославский район'],
                
 'Восточный административный округ':
 [
     'Богородское',
 'Вешняки',
 'Восточный',
 'Восточное Измайлово',
 'Гольяново',
 'Ивановское',
 'Измайлово',
 'Косино-Ухтомский',
 'Метрогородок',
 'Новогиреево',
 'Новокосино',
 'Перово',
 'Преображенское',
 'Северное Измайлово',
 'Соколиная Гора',
 'Сокольники'],
                
 'Юго-Восточный административный округ':
 [
     'Выхино-Жулебино',
 'Капотня',
 'Кузьминки',
 'Лефортово',
 'Люблино',
 'Марьино',
 'Некрасовка',
 'Нижегородский район',
 'Печатники',
 'Рязанский район',
 'Текстильщики',
 'Южнопортовый район'],
                
 'Южный административный округ':
 [
     'Бирюлёво Восточное',
 'Бирюлёво Западное',
 'Братеево',
 'Даниловский район',
 'Донской район',
 'Зябликово',
 'Москворечье-Сабурово',
 'Нагатино-Садовники',
 'Нагатинский Затон',
 'Нагорный район',
 'Орехово-Борисово Северное',
 'Орехово-Борисово Южное',
 'Царицыно',
 'Чертаново Северное',
 'Чертаново Центральное',
 'Чертаново Южное'],
                
 'Юго-Западный административный округ':
 [
     'Академический район',
 'Гагаринский район',
 'Зюзино',
 'Коньково',
 'Котловка',
 'Ломоносовский район',
 'Обручевский район',
 'Северное Бутово',
 'Тёплый Стан',
 'Черёмушки',
 'Южное Бутово',
 'Ясенево'],
                
 'Западный административный округ':
 [
     'Внуково',
 'Дорогомилово',
 'Крылатское',
 'Кунцево',
 'Можайский район',
 'Ново-Переделкино',
 'Очаково-Матвеевское',
 'Проспект Вернадского',
 'Раменки',
 'Солнцево',
 'Тропарёво-Никулино',
 'Филёвский Парк',
 'Фили-Давыдково'],
                
 'Северо-Западный административный округ':
 [
     'Куркино',
 'Митино',
 'Покровское-Стрешнево',
 'Северное Тушино',
 'Строгино',
 'Хорошёво-Мнёвники',
 'Щукино',
 'Южное Тушино'],
                
 'Зеленоградский административный округ':
 ['Матушкино',
 'Савёлки',
 'Старое Крюково',
 'Силино',
 'Крюково']
}

In [3]:
raion_list = list()
for okrug, inner_dict in moscow_okrug.items():
    for raion in inner_dict:
        raion_list.append((
            raion.replace("район", "").strip().lower(), okrug.lower()))
        
moscow_raion_okrug = pd.DataFrame(raion_list).rename({0: "raion", 1:"okrug"}, axis=1)
moscow_raion_okrug

,raion,okrug
0,арбат,центральный административный округ
1,басманный,центральный административный округ
2,замоскворечье,центральный административный округ
3,красносельский,центральный административный округ
4,мещанский,центральный административный округ
...,...,...
120,матушкино,зеленоградский административный округ
121,савёлки,зеленоградский административный округ
122,старое крюково,зеленоградский административный округ
123,силино,зеленоградский административный округ


In [4]:
nb = 'forAnalysis/boundary-polygon_Moscow.csv'
moscow = gpd.read_file(nb)
moscow = moscow.iloc[:, [1, -1]]
moscow["raion"] = copy.deepcopy(moscow.NAME)
moscow = moscow[moscow.raion.str.contains("район")]
moscow.raion = moscow.raion.str.replace("район", "").str.strip().str.lower()
moscow = moscow.rename({"NAME": "raion_name"}, axis=1)
moscow

,raion_name,geometry,raion
1,Басманный район,"POLYGON ((37.66592 55.75304, 37.66579 55.75303...",басманный
3,район Внуково,"MULTIPOLYGON (((37.24580 55.60191, 37.24656 55...",внуково
17,Одинцовский район,"MULTIPOLYGON (((37.17799 55.60788, 37.17961 55...",одинцовский
23,район Ивановское,"POLYGON ((37.82373 55.79020, 37.82422 55.79014...",ивановское
24,район Савёлки,"POLYGON ((37.23119 56.00462, 37.23540 56.00297...",савёлки
...,...,...,...
156,Донской район,"POLYGON ((37.58214 55.71099, 37.58227 55.71103...",донской
158,район Щукино,"POLYGON ((37.49907 55.78885, 37.49853 55.78878...",щукино
159,район Митино,"POLYGON ((37.39411 55.85443, 37.39456 55.85440...",митино
162,район Кунцево,"MULTIPOLYGON (((36.91270 55.74751, 36.91598 55...",кунцево


In [5]:
merged = moscow.merge(moscow_raion_okrug, how="right") # right only for moscow
merged = merged.iloc[:, [3, 0, 2, 1]]


# with open(f'{raion}, {okrug}.xlsx', 'w') as f:
#     f.write(merged)

In [6]:
merged.to_csv("forAnalysis/moscow.csv", index=False)

In [7]:
merged

,okrug,raion_name,raion,geometry
0,центральный административный округ,район Арбат,арбат,"POLYGON ((37.57413 55.74545, 37.57445 55.74633..."
1,центральный административный округ,Басманный район,басманный,"POLYGON ((37.66592 55.75304, 37.66579 55.75303..."
2,центральный административный округ,район Замоскворечье,замоскворечье,"POLYGON ((37.62392 55.73011, 37.62394 55.73020..."
3,центральный административный округ,Красносельский район,красносельский,"POLYGON ((37.68484 55.78081, 37.68335 55.77970..."
4,центральный административный округ,Мещанский район,мещанский,"POLYGON ((37.62150 55.75916, 37.62144 55.75929..."
...,...,...,...,...
120,зеленоградский административный округ,район Матушкино,матушкино,"POLYGON ((37.21243 55.98703, 37.21217 55.98704..."
121,зеленоградский административный округ,район Савёлки,савёлки,"POLYGON ((37.23119 56.00462, 37.23540 56.00297..."
122,зеленоградский административный округ,район Старое Крюково,старое крюково,"POLYGON ((37.20838 55.96985, 37.18882 55.97413..."
123,зеленоградский административный округ,район Силино,силино,"POLYGON ((37.19047 56.01351, 37.18995 56.01341..."


In [44]:
ekat = gpd.read_file("forAnalysis/boundary-polygon_Ekat.csv")

lst = []
for i in ["Верх-Исетский", "Академический", "Верх-Исетский", "Академический", "Железнодорожный",
 "Кировский", "Ленинский", "Октябрьский","Орджоникидзевский", "Чкаловский", "Орджоникидзевский", "Чкаловский"]:
    lst.append(i.lower())

result = pd.DataFrame()
for i in lst:
    
    x = ekat[(ekat.NAME.str.lower().str.contains(i)) &
             (ekat.ADMIN_L6.str.lower().str.contains("Екатеринбург".lower()))]
    result = pd.concat([result, x], join='outer', ignore_index=True).drop_duplicates()

result = result.iloc[:, [1, -1]].reset_index(drop=True).rename({"NAME": "raion_name"}, axis=1)
result.to_csv("forAnalysis/ekaterinburg.csv", index=False)

In [45]:
spb = gpd.read_file("forAnalysis/boundary-polygon_SPB.csv")
spb = spb[(data.NAME.str.lower().str.contains("район")) &
    ('Санкт-Петербург'==data.ADMIN_L4)]

spb = spb.iloc[:, [1, -1]].reset_index(drop=True).rename({"NAME": "raion_name"}, axis=1)
spb.to_csv("forAnalysis/spb.csv", index=False)

In [41]:
result

,NAME,geometry
0,Верх-Исетский район,"POLYGON ((60.60165 56.84262, 60.60229 56.84170..."
1,Железнодорожный район,"POLYGON ((60.51142 56.93974, 60.51039 56.93364..."
2,Кировский район,"POLYGON ((60.63278 56.84161, 60.63085 56.84139..."
3,Ленинский район,"POLYGON ((60.48775 56.76335, 60.49141 56.76641..."
4,Октябрьский район,"POLYGON ((60.62296 56.80988, 60.62623 56.81017..."
5,Орджоникидзевский район,"POLYGON ((60.51142 56.93974, 60.51283 56.93968..."
6,Чкаловский район,"POLYGON ((60.48775 56.76335, 60.49467 56.76080..."
